In [1]:
import json
import os
import numpy as np
        
from shapely.geometry import Polygon
import numpy as np
import time 
from shapely.validation import make_valid
import warnings
import cv2

warnings.filterwarnings(action='ignore')

import matplotlib.pyplot as plt

class Make_Polygon_without_overlap():
    def __init__(self, ori_path, save_path,json_count,img_dir):
        self.iou = 0.9
        self.ori_path = ori_path
        self.save_path = save_path
        self.json_count = json_count
        self.img_dir = img_dir
    
    def load_json(self,json_file) : 
        json_file_path = self.ori_path  + json_file
        print(f'START {json_file_path}')
        
        with open(json_file_path, 'r') as file:
            self.original_json_dict = json.load(file)
            new_points_list = []
            for seg_idx, j_dict in enumerate(self.original_json_dict['seg']) :
                seg_points = j_dict['points']
                points_dict = {}
                points_list = []

                for j_point in seg_points :
                    x_y = [j_point['x'],j_point['y']]
                    points_list.append(x_y)
                    
                

                points_list = np.array(points_list)
                points_list = points_list.reshape((-1, 2))
                
                # 선 제거 
                if points_list.shape[0] < 3 :
                    continue
                # Polygon 생성
                points_list = Polygon(points_list)
                # multi Polygon 분리
                points_list = make_valid(points_list)
                # Polygon type별로 dict 생성 (선종류는 버리고 polygon으로만 구성하기)
                if points_list.geom_type == 'GeometryCollection':
                    geometryCollection = list(points_list.geoms)

                    for geometry_points_list in geometryCollection:
                        g_type = geometry_points_list.geom_type
                        if g_type == 'MultiPolygon' : 
                            geometry_points_list = list(geometry_points_list.geoms)
                            for g_m_polygon in geometry_points_list:
                                points_dict = {}
                                box = {"left":[],"top":[],"width":[],"height":[]}
                                points_dict['box'] = box
                                points_dict['points'] = g_m_polygon
                                new_points_list.append(points_dict)
                        elif g_type == 'MultiLineString' : 
                            pass
                        elif g_type == 'LineString' :
                            pass
                        elif g_type == 'Polygon' : 
                            points_dict = {}
                            box = {"left":[],"top":[],"width":[],"height":[]}
                            points_dict['points'] = geometry_points_list
                            new_points_list.append(points_dict)
                        else : 
                            print('except GeometryCollection type : ', g_type)

                elif points_list.geom_type == 'MultiPolygon':
                    multipolygon = list(points_list.geoms)
                    for m_polygon in multipolygon:
                        points_dict = {}
                        box = {"left":[],"top":[],"width":[],"height":[]}
                        points_dict['box'] = box
                        points_dict['points'] = m_polygon
                        new_points_list.append(points_dict)

                elif points_list.geom_type == 'Polygon':
                    points_dict = {}
                    box = {"left":[],"top":[],"width":[],"height":[]}
                    points_dict['box'] = box
                    points_dict['points'] = points_list
                    new_points_list.append(points_dict)
                elif points_list.geom_type == 'MultiLineString' :
                    pass
                else : 
                    print('except points_list type',points_list.geom_type)


        return new_points_list
    
    
    def delete_overlap_points(self,new_points_list) : 
        del_point = []
        for idx_1, polygon_1_dict in enumerate(new_points_list) :
            polygon_1 = polygon_1_dict['points']
            for idx_2 in range(idx_1+1,len(new_points_list)):
                polygon_2 = new_points_list[idx_2]['points']
                intersect = polygon_1.intersection(polygon_2).area

                union = polygon_1.union(polygon_2).area
                if union == 0 :
                    continue
                iou = intersect / union

                if iou >= self.iou :
                    polygon_1_area = polygon_1.area
                    polygon_2_area = polygon_2.area

                    if polygon_1_area > polygon_2_area :
                        del_point.append(idx_2)
                    else :
                        del_point.append(idx_1)

        new_points_list_ori = new_points_list.copy()
        del_point.sort(reverse=True)
        print('total delete count',len(del_point))
        for del_key in del_point : 
            del new_points_list[del_key]

        return new_points_list
        
    def make_new_json(self,new_points_list,json_file) :       
        new_json_dict = dict()
        new_json_dict['image_name'] = self.original_json_dict['image_name']
        new_json_dict['seg'] = []

        for polygon_dict in new_points_list :
            new_points_list = []
            new_polygon_dict = {}
            
            polygon = polygon_dict['points']
            print('polygon',polygon)
            xywh = polygon.bounds
            xywh = [int(xywh[0]),int(xywh[1]),int(xywh[2]-xywh[0]),int(xywh[3]-xywh[1])]
            polygon = list(polygon.exterior.coords)

            new_polygon_dict['box'] = {"left":xywh[0],"top":xywh[1],"width":xywh[2],"height":xywh[3]}

            for x_y in polygon :
                x = int(x_y[0])
                y = int(x_y[1])
                p_dict = {'x': x, 'y': y}
                if p_dict in new_points_list :
                    pass
                else : 
                    new_points_list.append(p_dict)
                    
            new_polygon_dict['points'] = new_points_list
            new_json_dict['seg'].append(new_polygon_dict)
        
#         #json 파일로 저장
#         if not os.path.exists(self.save_path):
#             os.mkdir(self.save_path)
            
#         save_json_file_path = self.save_path  + json_file
        
#         with open(save_json_file_path, 'w', encoding='utf-8') as file:
#             json.dump(new_json_dict, file, indent="\t")
#         print(f'SAVE COMPLETE {save_json_file_path}')
        
    def get_file_list(self) : 
        fileEx = r'.json'
        json_list = [file for file in os.listdir(self.ori_path) if file.endswith(fileEx)]
        if self.json_count == 0 :
            return json_list
        else : 
            json_list = json_list[:self.json_count]
            return json_list
    def draw_poly(self,new_points_list,json_file):
        img_file = json_file[:-4] + 'jpg'
        img_path = self.img_dir + img_file
        print('img_path',img_path)
        image_ori = cv2.imread(img_path)
        image_ori_2 = cv2.imread(img_path)
        
        for polygon in new_points_list :
            polygon = polygon['points']
            polygon = list(polygon.exterior.coords)
            polygon = np.array(polygon)
            pts = np.array(polygon,np.int32)
            pts = pts.reshape((-1, 1, 2))

            color_np = np.asarray(np.random.choice(range(256), size=3), dtype=np.uint8)
            color_fill = (color_np[0].item(),color_np[1].item(),color_np[2].item())

            isClosed = True
            color = (255, 0, 0)
            thickness = 2
            line_image2 = cv2.polylines(image_ori, [pts], isClosed, color_fill, thickness)
            fill_image = cv2.fillPoly(image_ori_2, pts=[pts], color=color_fill)


        f = plt.figure()
        f.set_figwidth(20)
        f.set_figheight(15)
        plt.imshow(fill_image)
        plt.show()

        f = plt.figure()
        f.set_figwidth(20)
        f.set_figheight(15)
        plt.imshow(line_image2)
        plt.show()
        
    def main(self):
        json_list = self.get_file_list()
        
        for idx, json_file in enumerate(json_list) : 
            print(f'count : {idx+1}, total_file_count : {len(json_list)}, json_file : {json_file}')
            new_points_list = self.load_json(json_file)
            new_points_list = self.delete_overlap_points(new_points_list)
            self.draw_poly(new_points_list,json_file)
            self.make_new_json(new_points_list,json_file)
            print('_____________________________________________________________________________________')

        
if __name__ == '__main__':        
    original_josn_path = 'json/'
    new_josn_save_path = 'new_json/'
    img_dir = 'aihub_img/'
    json_count = 0
    m = Make_Polygon_without_overlap(original_josn_path, new_josn_save_path,json_count,img_dir)
    m.main()
    print('DONE.')

ModuleNotFoundError: No module named 'shapely'